### Prompt Length and Complexity Management
#### Overview
This tutorial explores techniques for managing prompt length and complexity when working with large language models (LLMs). We'll focus on two key aspects: balancing detail and conciseness in prompts, and strategies for handling long contexts.

#### Motivation
Effective prompt engineering often requires finding the right balance between providing enough context for the model to understand the task and keeping prompts concise for efficiency. Additionally, many real-world applications involve processing long documents or complex multi-step tasks, which can exceed the context window of LLMs. Learning to manage these challenges is crucial for building robust AI applications.

In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini")

print("Setup complete!")

Setup complete!


#### Balancing Detail and Conciseness

In [3]:
# Detailed prompt
detailed_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""Please provide a comprehensive explanation of {topic}. Include its definition, 
    historical context, key components, practical applications, and any relevant examples. 
    Also, discuss any controversies or debates surrounding the topic, and mention potential 
    future developments or trends."""
)

# Concise prompt
concise_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Briefly explain {topic} and its main importance."
)

topic = "artificial intelligence"

print("Detailed response:")
print(llm.invoke(detailed_prompt.format(topic=topic)).content)

print("\nConcise response:")
print(llm.invoke(concise_prompt.format(topic=topic)).content)

Detailed response:
### Definition of Artificial Intelligence (AI)

Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems. These processes include learning (the acquisition of information and rules for using it), reasoning (the use of rules to reach approximate or definite conclusions), problem-solving, perception, and language understanding. Essentially, AI enables machines to perform tasks that typically require human intelligence.

### Historical Context

The concept of AI dates back to ancient myths and stories about intelligent automatons, but formal developments began in the mid-20th century. Key milestones include:

1. **The Dartmouth Conference (1956)**: Often credited as the birthplace of AI as a field, organized by John McCarthy, Marvin Minsky, Nathaniel Rochester, and Claude Shannon. It aimed to explore whether machines could simulate any aspect of human intelligence.

2. **The Early Years**: The initi

#### Analysis of Prompt Balance

In [4]:
analysis_prompt = PromptTemplate(
    input_variables=["topic", "detailed_response", "concise_response"],
    template="""Compare the following two responses on {topic}:

Detailed response:
{detailed_response}

Concise response:
{concise_response}

Analyze the differences in terms of:
1. Information coverage
2. Clarity and focus
3. Potential use cases for each type of response

Then, suggest strategies for balancing detail and conciseness in prompts."""
)

detailed_response = llm.invoke(detailed_prompt.format(topic=topic)).content
concise_response = llm.invoke(concise_prompt.format(topic=topic)).content

analysis = llm.invoke(analysis_prompt.format(
    topic=topic,
    detailed_response=detailed_response,
    concise_response=concise_response
)).content

print(analysis)

### Analysis of Differences

1. **Information Coverage**:
   - **Detailed Response**: This response is comprehensive, covering a wide range of topics related to AI, including its definition, historical context, key components, practical applications, ethical controversies, and future trends. The response provides in-depth insights into the evolution of AI, important figures, and specific applications across various industries, making it valuable for those seeking a thorough understanding of the field.
   - **Concise Response**: The concise response presents a high-level overview of AI without delving into specifics. It encapsulates the essence of AI, mentions core technologies, and highlights the importance of AI in improving efficiency and decision-making. However, it lacks detailed historical context, specific use cases, ethical concerns, and future developments.

2. **Clarity and Focus**:
   - **Detailed Response**: While the detailed response is informative, it can be overwhelming 

### Strategies for Handling Long Contexts

#### 1. Chunking
Chunking involves breaking long texts into smaller, manageable pieces. Let's demonstrate this using a long text passage

In [1]:
# [A long passage about artificial intelligence, its history, applications, and future prospects...]

long_text = """
Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines that can simulate human cognitive processes.
The field of AI has a rich history dating back to the 1950s, with key milestones such as the development of the first neural networks and expert systems.
AI encompasses a wide range of subfields, including machine learning, natural language processing, computer vision, and robotics.
Practical applications of AI include speech recognition, image classification, autonomous vehicles, and medical diagnosis.
AI has the potential to revolutionize many industries, from healthcare and finance to transportation and entertainment.
However, there are ongoing debates and controversies surrounding AI, such as concerns about job displacement, bias in algorithms, and the ethical implications of autonomous systems.
Looking ahead, the future of AI holds promise for advancements in areas like explainable AI, AI ethics, and human-AI collaboration. 
The intersection of AI with other technologies like blockchain, quantum computing, and biotechnology will likely shape the future of the field.
But as AI continues to evolve, it is essential to consider the societal impact and ethical implications of these technologies.
One of the key challenges for AI researchers and developers is to strike a balance between innovation and responsibility, ensuring that AI benefits society as 
a whole while minimizing potential risks.
If managed effectively, AI has the potential to transform our world in ways we can only begin to imagine.
Though the future of AI is uncertain, one thing is clear: the impact of artificial intelligence will be profound and far-reaching.
"""

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

# Split the text into chunks
chunks = text_splitter.split_text(long_text)

print(f"Number of chunks: {len(chunks)}")
print(f"First chunk: {chunks[0][:200]}...")

NameError: name 'RecursiveCharacterTextSplitter' is not defined

#### 2. Summarization
Summarization can be used to condense long texts while retaining key information. Let's use LangChain's summarization chain to demonstrate this.

In [5]:
from langchain.docstore.document import Document

# Convert text chunks to Document objects
doc_chunks = [Document(page_content=chunk) for chunk in chunks]

# Load the summarization chain
chain = load_summarize_chain(llm, chain_type="map_reduce")

# Summarize the long text
summary_result = chain.invoke(doc_chunks)

print("Summary:")
print(summary_result['output_text'])

Summary:
Artificial intelligence (AI), a branch of computer science originating in the 1950s, aims to create machines that emulate human cognitive functions. Key areas include machine learning, natural language processing, and robotics, leading to applications like speech recognition and autonomous vehicles. While AI has the potential to transform industries, it also raises concerns about job displacement, algorithmic bias, and ethical issues. Future developments may focus on explainable AI, ethical considerations, and enhancing human-AI collaboration. Balancing innovation with responsibility is essential as AI evolves, with its impact expected to be profound and far-reaching.


In [6]:
def iterative_analysis(text, steps):
    """
    Perform iterative analysis on a given text.
    
    Args:
    text (str): The text to analyze.
    steps (list): List of analysis steps to perform.
    
    Returns:
    str: The final analysis result.
    """
    result = text
    for step in steps:
        prompt = PromptTemplate(
            input_variables=["text"],
            template=f"Analyze the following text. {step}\n\nText: {{text}}\n\nAnalysis:"
        )
        result = llm.invoke(prompt.format(text=result)).content
    return result

analysis_steps = [
    "Identify the main topics discussed.",
    "Summarize the key points for each topic.",
    "Provide a brief conclusion based on the analysis."
]

final_analysis = iterative_analysis(long_text, analysis_steps)
print("Final Analysis:")
print(final_analysis)

Final Analysis:
The text provides a comprehensive overview of artificial intelligence (AI), elucidating its definition, historical context, subfields, applications, and significant implications for various industries. It highlights the transformative potential of AI while addressing critical debates around issues such as job displacement, algorithmic bias, and ethical concerns. Furthermore, the text underscores the importance of responsible AI development, emphasizing the need for future innovations to consider societal impacts and ethical frameworks.

**Conclusion**: The analysis indicates that a balanced and ethical approach to AI development is crucial. As AI continues to evolve, stakeholders must navigate the challenges it presents while maximizing its benefits across different sectors. Addressing ethical concerns and societal implications will be vital in ensuring that AI technologies contribute positively to society.


In [7]:
tips_prompt = """
Based on the examples and strategies we've explored for managing prompt length and complexity, 
provide a list of 5 practical tips for developers working with large language models. 
Each tip should be concise and actionable.
"""

tips = llm.invoke(tips_prompt).content
print(tips)

Certainly! Here are five practical tips for developers working with large language models:

1. **Break Down Prompts:** Divide complex tasks into smaller, more manageable subtasks. This helps the model focus on one aspect at a time, leading to clearer and more accurate responses.

2. **Use Clear Instructions:** Frame prompts with explicit and straightforward language. Avoid ambiguity by specifying the desired format and length of the response to ensure the model understands your requirements.

3. **Iterative Refinement:** Start with a basic prompt and iteratively refine it based on model output. Adjust wording, add context, or alter the structure to improve the quality of responses.

4. **Experiment with Context:** Test variations in the context provided to the model. Including relevant background information can significantly influence the quality of the output, so find the right balance between context and prompt brevity.

5. **Monitor Token Usage:** Keep track of token limits for bot